In [ ]:
import sqlalchemy
sqlalchemy.__version__

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
                            self.name, self.fullname, self.password)
    


In [ ]:
Base.metadata.create_all(engine)


In [ ]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
ed_user.name

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

# Session = sessionmaker()

Session.configure(bind=engine)

session = Session()


In [ ]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

session.add(ed_user)

# ed_user.password = 'f8s7ccs'

session.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')])

session.commit()


In [ ]:
ed_user.id

In [ ]:
ed_user.name = 'Edwardo'
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

In [ ]:
session.rollback()

ed_user.name

In [ ]:
for instance in session.query(User).order_by(User.id):
    print(instance.id, instance.name)

In [ ]:
for name, fullname in session.query(User.name, User.fullname):
    print(name,fullname)
    

In [ ]:
for name, in session.query(User.name).\
            filter_by(fullname='Ed Jones'):
    print(name)
    

In [ ]:
for user in session.query(User).\
         filter(User.name.ilike('%ed%')).\
         filter(User.fullname=='Ed Jones'):
    print(user)

In [ ]:
for user in session.query(User).\
         filter(User.name.in_(['ed', 'wendy', 'jack'])):
    print(user)




In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

User.addresses = relationship(
    "Address", order_by=Address.id, back_populates="user")

Base.metadata.create_all(engine)

In [ ]:
jack = User(name='jack', fullname='Jack Bean', password='gjffdd')
jack.addresses

In [ ]:
jack.addresses = [
                Address(email_address='jack@google.com'),
                Address(email_address='j25@yahoo.com')]

In [ ]:
jack.addresses[1].user

In [ ]:
session.add(jack)
session.commit()

In [ ]:
jack = session.query(User).filter_by(name='jack').one()
jack

In [ ]:
from sqlalchemy import Table, Text
# association table
post_keywords = Table('post_keywords', Base.metadata,
    Column('post_id', ForeignKey('posts.id'), primary_key=True),
    Column('keyword_id', ForeignKey('keywords.id'), primary_key=True)
)

class BlogPost(Base):
    __tablename__ = 'posts'

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    headline = Column(String(255), nullable=False)
    body = Column(Text)

    # many to many BlogPost<->Keyword
    keywords = relationship('Keyword',
                            secondary=post_keywords,
                            back_populates='posts')

    def __init__(self, headline, body, author):
        self.author = author
        self.headline = headline
        self.body = body

    def __repr__(self):
        return "BlogPost(%r, %r, %r)" % (self.headline, self.body, self.author)


class Keyword(Base):
    __tablename__ = 'keywords'

    id = Column(Integer, primary_key=True)
    keyword = Column(String(50), nullable=False, unique=True)
    posts = relationship('BlogPost',
                         secondary=post_keywords,
                         back_populates='keywords')

    def __init__(self, keyword):
        self.keyword = keyword
        
        
        
BlogPost.author = relationship(User, back_populates="posts")
User.posts = relationship(BlogPost, back_populates="author", lazy="dynamic")

Base.metadata.create_all(engine)

wendy = session.query(User).\
                filter_by(name='wendy').\
                one()
post = BlogPost("Wendy's Blog Post", "This is a test", wendy)
session.add(post)


In [ ]:
post.keywords.append(Keyword('wendy'))
post.keywords.append(Keyword('firstpost'))

In [ ]:
session.query(BlogPost).\
            filter(BlogPost.keywords.any(keyword='firstpost')).\
            all()

In [25]:
def well_addresses():
    '''Generates a list of well address A1-H12'''
    letter = ["A","B","C","D","E","F","G","H"]
    number = ["1","2","3","4","5","6","7","8","9","10","11","12"]
    target_well = []
    temp_well = 0
    for n in number:
        for l in letter:
            temp_well = l + n
            target_well.append(temp_well)
    return target_well

class Well():
    def __init__(self,well_add,gene):
        print("made well {} with {}".format(well_add,gene))
        self.address = well_add
        self.gene = gene

class Plate():
    def __init__(self):
        print("made a plate")
        wells = well_addresses()
        self.counter = 0
        self.well_addresses = wells
        self.next_well = self.well_addresses[self.counter]
    
    def make_well(self,gene):
        print(self.well_addresses[self.counter])
        self.well_addresses[self.counter] = Well(self.next_well,gene)
        self.counter += 1
        self.next_well = self.well_addresses[self.counter]
        

    

genes = ['gene1','gene2','gene3']
plate1 = Plate()
for gene in genes:
    plate1.make_well(gene)
# plate1.A1
plate1.well_addresses[0].gene
plate1.well_addresses[0].address


# plate1.make_well('gene2')

made a plate
A1
made well A1 with gene1
B1
made well B1 with gene2
C1
made well C1 with gene3


'A1'

In [28]:
class Person:
    def __init__(self, first, last):
        self.firstname = first
        self.lastname = last
        print("init person")

    def Name(self):
        return self.firstname + " " + self.lastname

class Employee(Person):

    def __init__(self, first, last, staffnum):
        Person.__init__(self,first, last)
        self.staffnumber = staffnum
        print('init employee')

    def GetEmployee(self):
        return self.Name() + ", " +  self.staffnumber

x = Person("Marge", "Simpson")
y = Employee("Homer", "Simpson", "1007")

print(x.Name())
print(y.GetEmployee())
print(y.Name())

init person
init person
init employee
Marge Simpson
Homer Simpson, 1007
Homer Simpson
